# Whispers of the Heart - A daily reflection journal

Journaling allows individuals to reflect on their thoughts, feelings, and experiences, promoting self-awareness and personal growth. As journals creates a safe space for a person to freely express their thoughts and feelings, therapists often use journals as a therapeutic tool to gain deeper insights into their clients' thoughts, feelings, and experiences.

## Library Imports

In [74]:
# Library installations

%pip install matplotlib pandas numpy openai requests ipython jupyter python-dotenv opencv-python azure-storage-blob azure-identity

     ---------------------------------------- 0.0/77.0 kB ? eta -:--:--
     ---------------------------------------- 77.0/77.0 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/394.5 kB ? eta -:--:--
   ------------------------ --------------- 245.8/394.5 kB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 394.5/394.5 kB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/166.1 kB ? eta -:--:--
   ---------------------------------------- 166.1/166.1 kB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/193.4 kB ? eta -:--:--
   ---------------------------------------- 193.4/193.4 kB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.9 MB 17.6 MB/s eta 0:00:01
   ------------------ --------------------- 1.4/2.9 MB 17.2 MB/s eta 0:00:01
   ------------------------------ --------- 2.2/2.9 MB 17.3 MB/s eta 0:00:01
   ------------

In [1]:
# Loading the environment variables.
# Please check the config.env file and add the values.

from dotenv import load_dotenv
load_dotenv(dotenv_path="config.env")

True

In [2]:
#Importing the necessary libraries

import pandas as pd
from datetime import date
import os

from IPython.display import display, Markdown
import time

from openai import AzureOpenAI

from azure.storage.blob import BlobServiceClient, BlobSasPermissions, generate_blob_sas

import requests
import base64
from datetime import datetime, timedelta, timezone

In [3]:
# The API key import for the OpenAI model

OPENAI_ENDPOINT=os.getenv("OPENAI_ENDPOINT")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

## Initialising the variables

In [47]:
# Initialising the names of the files

data_folder = "data"
journal_file = "journal.txt"
analysis_file = "analysis.txt"

if "data" not in os.listdir():
  os.mkdir("data")

In [48]:
# Helper function to return the day of the week given the date

def day_of_the_week(date):
  weekday = date.weekday()
  days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
  return days[weekday]


# Initialising today's date

today = date.today()
date_display = str(day_of_the_week(today)) + " - " + str(today.strftime("%B %d, %Y"))

In [49]:
# Defining the prompt for the journal analysis - The context

journal_context = '''

You are part of a journalling app, where the person will write about his everyday experience and share his thoughts and feelings.
But you are more than any ordinary journal. You will track the emotions and thoughts of the person and make observations out of it.

Finally you will summarise the person's mental state for the day.

The response should not make any conclusive decision about what the person should do, as that is the therapist's job.
The response should be an observation of the person's mental well-being and making an analysis of it, to help the therapist come up with the decisions.

You can use the previous day's journal entry to have a progression in emotions, but the analysis and the summary has to be only of today's journal entry.

The structure of the response should be:

**Emotions**: <A list of atleast 3 emotions they are going through.>
**Possible thought patterns**: <A collection of 3-4 prominent thought patterns the person is having with a brief explanation.>

**Mental well-being scores:**
<A list of 10-15 mental states of the person, with two scores - confidence (how sure you are about the state of the person) and intensity (the strength of the emotion in the person).
Make both the scores out of 10. Include all major mental health states.
Make it look like a table.
Sort the states in the decreasing order of the confidence scores>

**Summary of the day:** <The summary should be informative to the therapist. Highlight the progresson of the person's mental state from the previous days to the current day. Keep the summary in 3-4 sentences/>

**Journal Excerpts:** <Top 3 excerpts of the journal that helped you make these analysis. Take only a small snippet (or a few short phrases) of the journal entry for each excerpt. Break down your reasoning step-by-step. Give clarifications or explanations for the analysis. DO NOT display the personal identification information of the person or anyone that they speak about. Keep the privacy intact. You can replace their information with placeholders.>

Previous days' journal entry:
{journal_yesterday}
Today's journal entry:
{journal_today}
'''

In [50]:
# Defining the prompt for the journal analysis - The journal entries

journal_prompt_template = '''

Previous days' journal entry:
{journal_yesterday}

Today's journal entry:
{journal_today}
'''

In [51]:
# Defining the prompt for the analysis of drawing

drawing_context = '''
You are a part of a journalling app, where the person will draw an image to convey his thoughts, feelings and mood.
But you are more than any ordinary journal. You will analyse and understand the emotions and thoughts of the person through the image.

You will also score the person's mental well-being on a few mental health 'symptoms' on a scale of 1 to 10 on how likely the person is in this state.
Finally you will summarise the person's mental state for the day.

The response should NOT make any conclusive decision about what the person should do, as that is the therapist's job.
The response should be an observation of the person's mental well-being and making an analysis of it, to help the therapist come up with the decisions.
The response should analyse the image and what the image is conveying and how the image is reflection of the person.

The structure of the response should be:

1. **Emotions**: <A list of atleast 3 emotions the image depicts.>
2. **Possible thought patterns**: <A collection of 3-4 prominent thought patterns the image is having with a brief explanation.>
3. **Mental well-being scores:**
<A list of 10-15 mental states as indicated by the image, with two scores - confidence (how sure you are about the state in the image) and intensity (the strength of the emotion in the image).
Make both the scores out of 10. Include all major mental health states.
Make it look like a table.
Sort the states in the decreasing order of the confidence scores>
4. **Summary of the day:** <The summary should be informative to the therapist. Keep the summary in 3-4 sentences/>
5. **Explanation:** <Explain your reasoning with detailed descriptions of the image. Break down your reasoning step-by-step. >
'''

In [52]:
# <NOT IN USE>
# Defining the prompt for the analysis of the video

video_context = '''
You are a part of a journalling app, where the person will send you a video to convey his thoughts, feelings and mood.
But you are more than any ordinary journal. You will analyse and understand the emotions and thoughts of the person who is listening to the video.

You will also score the person's mental well-being on a few mental health 'symptoms' on a scale of 1 to 10 on how likely the person is in this state.
Finally you will summarise the person's mental state for the day.

The response should NOT make any conclusive decision about what the person should do, as that is the therapist's job.
The response should be an observation of the person's mental well-being and making an analysis of it, to help the therapist come up with the decisions.

The structure of the response should be:

1. **Emotions**: <A list of atleast 3 emotions the person is in.>
2. **Possible thought patterns**: <A collection of 3-4 prominent thought patterns the person is having with a brief explanation.>
3. **Mental well-being scores:**
<A list of 10-15 mental states indicated by the person, with two scores - confidence (how sure you are about the state) and intensity (the strength of the state).
Make both the scores out of 10. Include all major mental health states.
Make it look like a table.
Sort the states in the decreasing order of the confidence scores>
4. **Summary of the day:** <The summary should be informative to the therapist. Keep the summary in 3-4 sentences/>
5. **Explanation:** <Explain your reasoning with detailed descriptions of the video and the person's state of mind. Break down your reasoning step-by-step. >
'''

In [53]:
# Calling the Azure OpenAI GPT 4 text model to 
# get the analysis for the journal entries.

def get_text_response(context, prompt):

    client = AzureOpenAI(
    azure_endpoint = OPENAI_ENDPOINT, 
    api_key=OPENAI_API_KEY,  
    api_version="2024-02-15-preview"
    )

    message_text = [
        {
            "role":"system",
            "content": context
        },
        {
            "role":"user",
            "content": prompt
        }
    ]

    completion = client.chat.completions.create(
        model="gpt-4-text", 
        messages = message_text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return completion.choices[0].message.content

In [54]:
# Calling the Azure OpenAI GPT 4 vision model to 
# get the analysis for the drawings.

def get_image_response(context, image_path):

    encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')
    headers = {
        "Content-Type": "application/json",
        "api-key": OPENAI_API_KEY,
    }

    # Payload for the request
    payload = {
    "messages": [
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": context
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}"
            }
            }
        ]
        }
    ],
    "temperature": 0.7,
    "top_p": 0.95,
    "max_tokens": 800
    }

    GPT4V_ENDPOINT = OPENAI_ENDPOINT+"/openai/deployments/gpt-4/chat/completions?api-version=2024-02-15-preview"

    # Send request
    try:
        response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    return response.json()['choices'][0]['message']['content']

In [106]:
# <NOT IN USE>
# Uploading the videos to the Azure Blob Storage
# for the GPT 4 vision model to analyze.

# def upload_video_to_storage(video_path):

#     # Set environment variables
#     STORAGE_ACCOUNT_NAME = os.getenv("STORAGE_ACCOUNT_NAME")
#     STORAGE_ACCOUNT_KEY = os.getenv("STORAGE_ACCOUNT_KEY")
#     CONTAINER_NAME = os.getenv("CONTAINER_NAME")
#     VIDEO_FILE_PATH = video_path

#     video_name = video_path.split('/')[-1]

#     # Create a blob service client
#     blob_service_client = BlobServiceClient.from_connection_string(
#         f"DefaultEndpointsProtocol=https;AccountName={STORAGE_ACCOUNT_NAME};AccountKey={STORAGE_ACCOUNT_KEY};BlobEndpoint=https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/"
#     )

#     # Create a container client
#     container_client = blob_service_client.get_container_client(CONTAINER_NAME)

#     # Upload the video file
#     with open(VIDEO_FILE_PATH, "rb") as data:
#         blob_client = container_client.get_blob_client(video_name)
#         blob_client.upload_blob(data, overwrite=True)
    
#     start_time = datetime.now(timezone.utc)
#     expiry_time = start_time + timedelta(days=1)

#     sas_token = generate_blob_sas(
#         account_name=blob_client.account_name,
#         container_name=blob_client.container_name,
#         blob_name=blob_client.blob_name,
#         account_key=STORAGE_ACCOUNT_KEY,
#         permission=BlobSasPermissions(read=True),
#         expiry=expiry_time,
#         start=start_time
#     )

#     # Construct the SAS URL
#     sas_url = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_client.container_name}/{video_name}?{sas_token}"

#     return sas_url

In [99]:
# <NOT IN USE>
# Calling the Azure OpenAI GPT 4 VISION model to 
# get the analysis for the uploaded videos.

# def get_video_response(context, video_sas_url):
    
#     GPT_4V_ENDPOINT = os.getenv("GPT_4V_ENDPOINT")
#     GPT_4V_KEY = os.getenv("GPT_4V_KEY")

#     VISION_API_ENDPOINT = os.getenv("VISION_API_ENDPOINT")
#     VISION_API_KEY = os.getenv("VISION_API_KEY")

#     ## ingest the video
#     VIDEO_FILE_SAS_URL = video_sas_url
#     VIDEO_INDEX_NAME = "Journal_analysis_video"
#     VIDEO_DOCUMENT_ID = "AOAIChatDocument"

#     def create_video_index(vision_api_endpoint, vision_api_key, index_name):
#         url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}?api-version=2024-05-06-preview"
#         headers = {"Ocp-Apim-Subscription-Key": vision_api_key, "Content-Type": "application/json"}
#         data = {
#             'metadataSchema': {
#                 'fields': [
#                 {
#                     'name': 'cameraId',
#                     'searchable': False,
#                     'filterable': True,
#                     'type': 'string'
#                 },
#                 {
#                     'name': 'timestamp',
#                     'searchable': False,
#                     'filterable': True,
#                     'type': 'datetime'
#                 }
#                 ]
#             },
#             'features': [
#                 {
#                 'name': 'vision',
#                 'domain': 'surveillance'
#                 },
#                 {
#                 'name': 'speech'
#                 }
#             ]
#         }
#         response = requests.put(url, headers=headers, json=data)
#         return response

#     def add_video_to_index(vision_api_endpoint, vision_api_key, index_name, video_url, video_id):
#         url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions/my-ingestion?api-version=2023-05-01-preview"
#         headers = {"Ocp-Apim-Subscription-Key": vision_api_key, "Content-Type": "application/json"}
#         data = {
#             'videos': [{'mode': 'add', 'documentId': video_id, 'documentUrl': video_url}]
#         }
#         response = requests.put(url, headers=headers, json=data)
#         return response

#     def wait_for_ingestion_completion(vision_api_endpoint, vision_api_key, index_name, max_retries=30):
#         url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions?api-version=2023-05-01-preview"
#         headers = {"Ocp-Apim-Subscription-Key": vision_api_key}
#         retries = 0
#         while retries < max_retries:
#             time.sleep(10)
#             response = requests.get(url, headers=headers)
#             if response.status_code == 200:
#                 state_data = response.json()
#                 if state_data['value'][0]['state'] == 'Completed':
#                     print(state_data)
#                     print('Ingestion completed.')
#                     return True
#                 elif state_data['value'][0]['state'] == 'Failed':
#                     print(state_data)
#                     print('Ingestion failed.')
#                     return False
#             retries += 1
#         return False


#     # Step 1: Create an Index
#     response = create_video_index(VISION_API_ENDPOINT, VISION_API_KEY, VIDEO_INDEX_NAME)
#     print(response.status_code, response.text)

#     # Step 2: Add a video file to the index
#     response = add_video_to_index(VISION_API_ENDPOINT, VISION_API_KEY, VIDEO_INDEX_NAME, VIDEO_FILE_SAS_URL, VIDEO_DOCUMENT_ID)
#     print(response.status_code, response.text)

#     # Step 3: Wait for ingestion to complete
#     if not wait_for_ingestion_completion(VISION_API_ENDPOINT, VISION_API_KEY, VIDEO_INDEX_NAME):
#         print("Ingestion did not complete within the expected time.")


#     ## Chat with GPT-4V

#     headers = {
#         "Content-Type": "application/json",
#         "api-key": GPT_4V_KEY,
#     }

#     # Payload for the request
#     payload = {
#         "dataSources": [
#             {
#                 "type": "AzureComputerVisionVideoIndex",
#                 "parameters": {
#                     "computerVisionBaseUrl": f"{VISION_API_ENDPOINT}/computervision",
#                     "computerVisionApiKey": VISION_API_KEY,
#                     "indexName": VIDEO_INDEX_NAME,
#                     "videoUrls": [VIDEO_FILE_SAS_URL]
#                 }
#             }
#         ],
#         "enhancements": {
#             "video": {
#                 "enabled": True
#             }
#         },
#         "messages": [
#         {
#             "role": "system",
#             "content": [
#                 {
#                         "type": "text",
#                         "text": context
#                 }
#             ]
#         },
#         {
#             "role": "user",
#             "content": [
#                 {
#                         "type": "acv_document_id",
#                         "acv_document_id": VIDEO_DOCUMENT_ID
#                 }
#             ]
#         }
#     ],
#         "temperature": 0.7,
#         "top_p": 0.95,
#         "max_tokens": 800
#     }

#     # Send request
#     try:
#         response = requests.post(GPT_4V_ENDPOINT, headers=headers, json=payload)
#         response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
#     except requests.RequestException as e:
#         raise SystemExit(f"Failed to make the request. Error: {e}")

#     # Handle the response as needed (e.g., print or process)
#     return response.json()['choices'][0]['message']['content']

In [56]:
# Writing all the answers into a file.
# This file will be accessible only for the person.
# The Therapist view will NOT have access to this file.

def writing_entry_to_file(entry, file_name):
  with open(os.path.join(data_folder, file_name), "a") as f:
    f.write(entry)

 ## Journal View

This is what the journalling person views in the app.

In [39]:
# We are taking the inputs for the journal here.
# The person can use the input text boxes to enter his thoughts from the day

display(Markdown("## Daily Reflection Journal\n"))

# Printing out today's date

display(Markdown(date_display))

# We have given a few thought-provoking questions to let the person reflect on his day.
# The question can be skipped by just pressing the Enter key without writing anything.
# Make sure to answer at least one question.
# (We decided not to add the force input here. But if we develop an app over this we can
# make sure to include this feature).

experience = input("\nWhat emotions did I experience today and why?\n")
lessons = input("\nDid I learn any insights or lessons from the day?\n")
obstacles = input("\nWhat challenges or obstacles did I face today, and how did I approach them?\n")
reflections = input("\nWhat are the key reflections from the day?\n")
extra = input("\nDo you want to share anything else? Write a poem? Have a quote you relate to? Or just want to scribble some text?\n")

drawing = input("\nDid you draw anything today? If yes, share the file path of the drawing.\n")
video = input("\nWould you like to share a video? If yes, share the file path of the video.\n")

## Daily Reflection Journal


Monday - May 06, 2024

In [40]:
# Format to write into the journal files.

journal_entry = f'''
----------------------------------------------------------
{date_display}

What emotions did I experience today and why?: {experience}
Did I learn any insights or lessons from the day?: {lessons}
What challenges or obstacles did I encounter today, and how did I approach them?: {obstacles}
What are the key reflections from the day?: {reflections}
Do you want to share anything else?: {extra}

Did you draw anything today? If yes, share only the file path of the drawing: {drawing}
Would you like to share a video? If yes, share the file path of the video: {video}
'''

writing_entry_to_file(journal_entry, journal_file)

In [41]:
print(journal_entry)


----------------------------------------------------------
Monday - May 06, 2024

What emotions did I experience today and why?: 
Did I learn any insights or lessons from the day?: 
What challenges or obstacles did I encounter today, and how did I approach them?: 
What are the key reflections from the day?: 
Do you want to share anything else?: 

Did you draw anything today? If yes, share only the file path of the drawing: 
Would you like to share a video? If yes, share the file path of the video: 



## Analysis

This is the part where GenAI comes into picture

In [21]:
# Fetch the journal entries from the files.

def get_entry(file_name):
  if file_name not in os.listdir(data_folder):
    return "", ""

  with open(os.path.join(data_folder, file_name), "r") as f:
    text_in_file = f.read()

  text_entries = text_in_file.split('----------------------------------------------------------')

  if len(text_entries) < 2:
    return text_entries[-1], ""

  return text_entries[-1], text_entries[-2]

In [57]:
# Get the analysis of the text journal.

def get_journal_analysis(journal_today, journal_yesterday):

  # journal_today_parts = journal_today.split('\n\n')
  journal_yesterday_parts = journal_yesterday.split('\n\n')

  journal_yesterday = journal_yesterday_parts[1] if len(journal_yesterday_parts) > 1 else ''

  journal_prompt = journal_prompt_template.format(journal_yesterday=journal_yesterday, journal_today=journal_today)

  response = get_text_response(context=journal_context, prompt=journal_prompt)

  journal_analysis = f'''
----------------------------------------------------------
### {date_display}

## Analysis of the journal

{response}

'''

  return journal_analysis

In [58]:
# Get the analysis of the drawings.

def get_drawing_analysis(journal_today):

    journal_today_parts = journal_today.split('\n')
    image_name = journal_today_parts[-3].split(':')[-1].replace(' ', '').replace('\n', '')

    if image_name == '' or image_name is None:
        return ""

    response = get_image_response(context=drawing_context, image_path=image_name)

    drawing_analysis = f'''

    ## Analysis of the drawing

    {response}

    '''

    return drawing_analysis

In [102]:
# <NOT IN USE>
# Get the analysis of the videos.

# def get_video_analysis(journal_today):

#     journal_today_parts = journal_today.split('\n')
#     video_name = journal_today_parts[-2].split(':')[-1].replace(' ', '').replace('\n', '')

#     if video_name == '' or video_name is None:
#         return ""

#     video_sas_url = upload_video_to_storage(video_path=video_name)
#     print(video_sas_url)
#     response = get_video_response(context=drawing_context, video_sas_url=video_sas_url)

#     video_analysis = f'''

#     ## Analysis of the video

#     {response.text}

#     '''

#     return video_analysis

In [114]:
journal_today, journal_yesterday = get_entry(journal_file)

journal_analysis = get_journal_analysis(journal_today=journal_today, journal_yesterday=journal_yesterday)
drawing_analysis = get_drawing_analysis(journal_today=journal_today)
#video_analysis = get_video_analysis(journal_today=journal_today)

# analysis_entry = journal_analysis + drawing_analysis + video_analysis
analysis_entry = journal_analysis + drawing_analysis
writing_entry_to_file(analysis_entry, analysis_file)

In [115]:
display(Markdown(analysis_entry))


----------------------------------------------------------
### Monday - May 06, 2024

## Analysis of the journal

**Emotions**: Frustration, Exhaustion, Resignation

**Possible thought patterns**:
- Perseverance despite obstacles: The individual is expressing a notion of pushing through challenges with a hint of sarcasm.
- Need for self-care: Recognizing the importance of sleep suggests an awareness of self-care necessities.
- Sense of humor as a coping mechanism: The use of humor when dealing with stress indicates a possible coping strategy.

**Mental well-being scores:**

| Mental State               | Confidence Score | Intensity Score |
|----------------------------|------------------|-----------------|
| Frustration                | 9                | 8               |
| Exhaustion                 | 9                | 7               |
| Stress                     | 8                | 7               |
| Humor                      | 8                | 5               |
| Need for self-care         | 8                | 6               |
| Resignation                | 7                | 6               |
| Patience (lack of)         | 7                | 6               |
| Hope                       | 6                | 4               |
| Determination              | 6                | 5               |
| Sleep Deprivation          | 9                | 7               |
| Concentration Difficulties | 8                | 6               |
| Hunger                     | 7                | 5               |
| Overwhelm                  | 7                | 5               |

**Summary of the day:** The individual's journal reflects a day marked by frustration and exhaustion, likely stemming from professional challenges and a lack of sleep. There's an underlying tone of humor and resignation, suggesting a coping mechanism to deal with the stress. The summary indicates a need for increased self-care, particularly in prioritizing rest and managing work-related stress.

**Journal Excerpts:**

1. "I'm really pissed off."
   - This phrase indicates a high level of frustration, possibly linked to the challenges faced with their work tasks.
   
2. "And sleepy and tired."
   - These words highlight exhaustion, both physical and mental, which is often exacerbated by insufficient sleep.
   
3. "Trust in the process and something will happen XD"
   - This excerpt, with its use of humor ('XD'), suggests an attempt to lighten the mood or a coping strategy to deal with the current difficulties. It may also indicate a faint sense of hope or optimism amidst the struggles.



    ## Analysis of the drawing

    1. **Emotions**: Support, companionship, trust.

2. **Possible thought patterns**: 
   - **Seeking or offering support**: The act of one figure helping another up a steep incline suggests thoughts around seeking or offering support.
   - **Companionship and connection**: The presence of animals following the human figures indicates thoughts about companionship and the importance of non-verbal connections.
   - **Overcoming obstacles**: The uphill path they are navigating implies thoughts related to overcoming challenges or obstacles in one's path.

3. **Mental well-being scores:**

   | Mental State          | Confidence Score | Intensity Score |
   |-----------------------|------------------|-----------------|
   | Supportiveness        | 9                | 7               |
   | Social Connectedness  | 8                | 6               |
   | Resilience            | 8                | 6               |
   | Trust                 | 7                | 5               |
   | Anxiety               | 3                | 2               |
   | Depression            | 3                | 2               |
   | Isolation             | 2                | 1               |
   | Stress                | 4                | 3               |
   | Optimism              | 6                | 4               |
   | Self-Esteem           | 5                | 3               |
   | Hopefulness           | 6                | 4               |
   | Determination         | 7                | 5               |
   | Vulnerability         | 4                | 3               |
   | Contentment           | 6                | 4               |
   | Loneliness            | 1                | 1               |

4. **Summary of the day:** The image suggests a sense of mutual support and companionship, indicating positive social connections. The uphill journey implies resilience and overcoming challenges. The presence of trust and a forward-moving direction suggests optimism and determination in the face of obstacles.

5. **Explanation:** 
   - The two human figures with blurred faces are depicted in an act of helping each other up a slope, symbolizing support and trust. This scene is indicative of a sense of camaraderie and mutual aid, which is important for emotional well-being.
   - The animals, which appear to be following the figures loyally, add to the sense of companionship and imply a bond that goes beyond verbal communication, hinting at the person's appreciation for non-human connections.
   - The uphill path is a classic metaphor for challenges or struggles. The act of climbing together suggests resilience and the ability to face and overcome difficulties, implying a sense of hope and determination.
   - The image does not convey intense negative emotions such as anxiety, depression, or loneliness; rather, it leans towards positive aspects such as optimism and contentment, as indicated by the tranquil forest setting and the light that seems to guide their way.

    

## Therapist View

This is what the therapist will see about the person.

In [116]:
# Fetch the analysis entry from the files.

def get_analysis_entry(file_name, days_from_today = 0):
  if file_name not in os.listdir(data_folder):
    return [""]

  with open(os.path.join(data_folder, file_name), "r") as f:
    text_in_file = f.read()

  text_entries = text_in_file.split('----------------------------------------------------------')
  text_entries = text_entries[1:]

  if days_from_today < 0:
    return "Please enter a positive number (or 0 for today)"

  if len(text_entries) < days_from_today + 1:
    return "No Entry"

  return text_entries[-1-days_from_today]

In [117]:
entry_day = input("Please enter the day of the analysis you want to see. 0 - today, 1 - yesterday, and so on...\n")
analysis_of_the_day = get_analysis_entry(analysis_file, days_from_today=int(entry_day))

In [118]:
display(Markdown(analysis_of_the_day))


### Monday - May 06, 2024

## Analysis of the journal

**Emotions**: Frustration, Exhaustion, Resignation

**Possible thought patterns**:
- Perseverance despite obstacles: The individual is expressing a notion of pushing through challenges with a hint of sarcasm.
- Need for self-care: Recognizing the importance of sleep suggests an awareness of self-care necessities.
- Sense of humor as a coping mechanism: The use of humor when dealing with stress indicates a possible coping strategy.

**Mental well-being scores:**

| Mental State               | Confidence Score | Intensity Score |
|----------------------------|------------------|-----------------|
| Frustration                | 9                | 8               |
| Exhaustion                 | 9                | 7               |
| Stress                     | 8                | 7               |
| Humor                      | 8                | 5               |
| Need for self-care         | 8                | 6               |
| Resignation                | 7                | 6               |
| Patience (lack of)         | 7                | 6               |
| Hope                       | 6                | 4               |
| Determination              | 6                | 5               |
| Sleep Deprivation          | 9                | 7               |
| Concentration Difficulties | 8                | 6               |
| Hunger                     | 7                | 5               |
| Overwhelm                  | 7                | 5               |

**Summary of the day:** The individual's journal reflects a day marked by frustration and exhaustion, likely stemming from professional challenges and a lack of sleep. There's an underlying tone of humor and resignation, suggesting a coping mechanism to deal with the stress. The summary indicates a need for increased self-care, particularly in prioritizing rest and managing work-related stress.

**Journal Excerpts:**

1. "I'm really pissed off."
   - This phrase indicates a high level of frustration, possibly linked to the challenges faced with their work tasks.
   
2. "And sleepy and tired."
   - These words highlight exhaustion, both physical and mental, which is often exacerbated by insufficient sleep.
   
3. "Trust in the process and something will happen XD"
   - This excerpt, with its use of humor ('XD'), suggests an attempt to lighten the mood or a coping strategy to deal with the current difficulties. It may also indicate a faint sense of hope or optimism amidst the struggles.



    ## Analysis of the drawing

    1. **Emotions**: Support, companionship, trust.

2. **Possible thought patterns**: 
   - **Seeking or offering support**: The act of one figure helping another up a steep incline suggests thoughts around seeking or offering support.
   - **Companionship and connection**: The presence of animals following the human figures indicates thoughts about companionship and the importance of non-verbal connections.
   - **Overcoming obstacles**: The uphill path they are navigating implies thoughts related to overcoming challenges or obstacles in one's path.

3. **Mental well-being scores:**

   | Mental State          | Confidence Score | Intensity Score |
   |-----------------------|------------------|-----------------|
   | Supportiveness        | 9                | 7               |
   | Social Connectedness  | 8                | 6               |
   | Resilience            | 8                | 6               |
   | Trust                 | 7                | 5               |
   | Anxiety               | 3                | 2               |
   | Depression            | 3                | 2               |
   | Isolation             | 2                | 1               |
   | Stress                | 4                | 3               |
   | Optimism              | 6                | 4               |
   | Self-Esteem           | 5                | 3               |
   | Hopefulness           | 6                | 4               |
   | Determination         | 7                | 5               |
   | Vulnerability         | 4                | 3               |
   | Contentment           | 6                | 4               |
   | Loneliness            | 1                | 1               |

4. **Summary of the day:** The image suggests a sense of mutual support and companionship, indicating positive social connections. The uphill journey implies resilience and overcoming challenges. The presence of trust and a forward-moving direction suggests optimism and determination in the face of obstacles.

5. **Explanation:** 
   - The two human figures with blurred faces are depicted in an act of helping each other up a slope, symbolizing support and trust. This scene is indicative of a sense of camaraderie and mutual aid, which is important for emotional well-being.
   - The animals, which appear to be following the figures loyally, add to the sense of companionship and imply a bond that goes beyond verbal communication, hinting at the person's appreciation for non-human connections.
   - The uphill path is a classic metaphor for challenges or struggles. The act of climbing together suggests resilience and the ability to face and overcome difficulties, implying a sense of hope and determination.
   - The image does not convey intense negative emotions such as anxiety, depression, or loneliness; rather, it leans towards positive aspects such as optimism and contentment, as indicated by the tranquil forest setting and the light that seems to guide their way.

    